Evaluating Generative models is difficult

1. Human Evaluation
2. Test Suites
3. Elo Ranking

Human Evaluation is most reliable

Good test data is crucial
    
    1. High-quality

    2. Accurate
    
    3. Generalized

    4. Not seen in training data

Elo comparison also popular

looking almost like a A-B test between multiple models or tournament across 

ELO rankings are used in chess specifically and so this is one way to understand which model performing well or not

Common suite is bunch of different evaluation methods. Taking bunch of different evaluation methods and averaging them all together to rank models.

EleutherAI :- it's set of different benchmarks put together.

Common LLM benchmarks:- 

1. ARC:-  a set of grade-school questions.
2. HellaSwag:-  test of common sense.
3. MMLU:- multitask metric covering elementary math, US history, computer science, law and more.
4. TruthfulQA:- measures models propensity to reproduce falsehoods commonly found online. 

Free willy model finetuned on top of llama-2 model using orca method

The ORCA Method in the context of Generative AI typically refers to a technique for improving large language model (LLM) performance by teaching smaller models to emulate the reasoning of larger, more capable models.

What is the ORCA Method?

ORCA stands for "Open-Ranked Chain-of-Thought Alignment" or similar variants depending on the paper or implementation. 

It was popularized in 2023 by Microsoft in their paper titled “Orca: Progressive Learning from Complex Explanation Traces of GPT-4”.

Purpose of the ORCA Method

The main goal of the ORCA method is to:

Train smaller language models (like 13B or even 7B parameter models)

To learn from larger LLMs like GPT-4

By focusing not just on final answers, but on the reasoning steps (chain-of-thought) behind those answers

This approach enhances the reasoning and instruction-following abilities of smaller models.

| Feature | Description |
|-|-|
| Teacher Model | Uses a powerful LLM (e.g., GPT-4) as the "teacher"|
| Student Model | Smaller LLM learns by imitating the teacher's reasoning process |
| Chain-of-Thought Tracing | Includes intermediate steps, not just the final answer |
| Progressive Learning | Starts simple, then adds more complex reasoning traces over time |
| Better Efficiency | Smaller models become much more capable without massive computational cost |

model | Average | ARC | HellaSwag | MMLU | TruthfuQA |
|-|-|-|-|-|-|
llama-2| 67.3 | 67.3 | 87.3 | 69.8 | 44.9 |
FreeWilly2 | 71.4 | 71.1 | 86.4 | 68.2 | 59.4 |
FreeWilly1 | 68.7 | 68.2 | 85.9 | 64.8 | 55.8 |

Error Analysis :- Other method for evaluating your model, 

It categorize the errors so that you understand the types of errors that are very common, and going after very common errors and very catastrophic errors first.

It usually requires you to train your model first beforehand, but for fine-tuning model its already pre-trained  so you can already perform error analysis before even fine-tuning the model.

This helps you understand and characterized how your base model is doing, so that you know what kind of data will give it the biggest lift for fine-tuning.

Categories 

1. Misspelling.

2. Length (Too -long)

3. Repetitive.

Error Analysis 

1. Understand the base model behavior before fine-tuning.

2. Categorize errors: iterate an data to fix these problems in data space


Category | Example with Problem | Example Fixed |
|-|-|-|
Misspelling | "Your kidney is healthy, your lever is sick. Go get your lever checked.| "Your Kidney is healthy. but your liver is sick"|
Too Long| "Diabetes is less likely when you eat healthy diet, because eating a healthy diet makes diabetes less likely , making..."| "Diabetes is less likely when you eat a healthy diet."|
Repetitve|"Medical LLm can save healthcare workers time and money and time and money and time and money."|"Medical LLM can save healthcare workers time and money."|
            
            

Can use other llm to compare/grade 2 outputs, can also use embedding so you can embedded actual answer and embed the generated answer and see how close they are in distance.

In [1]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import logging
import difflib
import pandas as pd

import transformers
import datasets
import torch

from tqdm import tqdm  #adds progress bars to iterations
# from utilities import *
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
logger = logging.getLogger(__name__)
global_config = None

In [4]:
dataset = datasets.load_dataset("lamini/lamini_docs")
test_dataset = dataset["test"]

In [17]:
print(test_dataset[0]["question"])
print(test_dataset[0]["answer"])

Can Lamini generate technical documentation or user manuals for software projects?
Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.


In [10]:
# pull out the fine tuned model from hugging face

model_name = "lamini/lamini_docs_finetuned"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [11]:
model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [12]:
# Let use exact match evaluation matrix

def is_exact_match(a,b):
    return a.strip()==b.strip()

In [13]:
# Run model in evaluation model so that things like dropout is disabled.
model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [19]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_token=100):
    #Tokenize
    
    tokenizer.pad_token = tokenizer.eos_token
    input_ids = tokenizer.encode(
        text,
        return_tensors = "pt",
        truncation = True,
        max_length=max_input_tokens
    )
    
    #Generate
    device = model.device
    generated_tokens_with_prompt = model.generate(
        input_ids = input_ids.to(device),
        max_length = max_output_token
    )
    
    #Decode
    generated_tokens_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)
    
    #Strip
    generated_text_answer = generated_tokens_with_prompt[0][len(text):]
    
    return generated_text_answer

The max_input_tokens parameter is used when working with language models (like GPT or other LLMs) to limit the number of tokens the model can receive as input. It’s important for controlling cost, performance, and avoiding errors due to exceeding model limits.



The context window size (also called context length) of a language model refers to the maximum number of tokens the model can "see" or process at once. It defines how much input (and output) the model can handle in a single pass.


How Context Window Is Used
If a model has a context window of 4,096 tokens, and:

Your prompt = 2,000 tokens

You request an output = 1,500 tokens

Total = 3,500 tokens ← OK (under the limit)

1. Context Window

Definition: The total number of tokens (input + output) a language model can handle at one time.

Set by the model, not the user.

Varies by model — for example:

GPT-3.5: 4,096 tokens

GPT-4 Turbo: 128,000 tokens

2. max_input_tokens

Definition: A user-defined limit on how many input tokens you're allowing the model to receive.

It's a subset of the context window.

Used for safety, consistency, or performance.

In [20]:
test_question = test_dataset[0]["question"]
generated_answer = inference(test_question, model, tokenizer)
print(test_question)
print(generated_answer)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Can Lamini generate technical documentation or user manuals for software projects?
Yes, Lamini can generate technical documentation or user manuals for software projects. This can be achieved by providing a prompt for a specific technical question or question to the LLM Engine, or by providing a prompt for a specific technical question or question. Additionally, Lamini can be trained on specific technical questions or questions to help users understand the process and provide feedback to the LLM Engine. Additionally, Lamini


In [21]:
answer = test_dataset[0]["answer"]
print(answer)

Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.


In [22]:
exact_match = is_exact_match(generated_answer,answer)
print(exact_match)

False


Another option will be to take this output and generated answer and ask other LLM to tell how close these outputs are really

you can also use embedding to generate embedding of both generated and actual answer and check how close the embeddings are.

lets run it on entire dataset

In [23]:
n=10 # will run on 10 examples
metrics = {'exact_matches': []}
predictions =[]

for i, item in tqdm(enumerate(test_dataset)):
    print("i Evaluating: "+ str(item))
    question = item['question']
    answer = item['answer']
    
    try:
        predicted_answer = inference(question, model, tokenizer)
    except:
        continue
    predictions.append([predicted_answer, answer])
    
    exact_match = is_exact_match(generated_answer,answer)
    metrics['exact_matches'].append(exact_match)
    
    if i > n and n != -1:
        break
    
print('Number of exact matches: ', sum(metrics['exact_matches']))


0it [00:00, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


i Evaluating: {'question': 'Can Lamini generate technical documentation or user manuals for software projects?', 'answer': 'Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.', 'input_ids': [5804, 418, 4988, 74, 6635, 7681, 10097, 390, 2608, 11595, 84, 323, 3694, 6493, 32, 4374, 13, 418, 4988, 74, 476, 6635, 7681, 10097, 285, 2608, 11595, 84, 323, 3694, 6493, 15, 733, 4648, 3626, 3448, 5978, 5609, 281, 2794, 2590, 285, 44003, 10097, 326, 310, 3477, 281, 2096, 323, 1097, 7681, 285, 1327, 14, 48746, 4212, 15, 831, 476, 5321, 12259, 247, 1534, 2408, 273, 673, 285, 3434, 275, 6153, 10097, 13, 6941, 731, 281, 2770, 327, 643, 7794, 273, 616, 6493, 1

1it [00:00,  1.10it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


i Evaluating: {'question': 'How do I include my API key in the Authorization HTTP header?', 'answer': 'The Authorization HTTP header should include the API key in the following format: Authorization: Bearer <YOUR-KEY-HERE>.', 'input_ids': [2347, 513, 309, 2486, 619, 8990, 2234, 275, 253, 10360, 1320, 17607, 10478, 32, 510, 10360, 1320, 17607, 10478, 943, 2486, 253, 8990, 2234, 275, 253, 1563, 5981, 27, 10360, 1320, 27, 2325, 12287, 654, 58, 11862, 14, 13888, 14, 41, 8147, 13208], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [2347, 513, 309, 2486, 619, 8990, 2234, 275, 253, 10360, 1320, 17607, 10478, 32, 510, 10360, 1320, 17607, 10478, 943, 2486, 253, 8990, 2234, 275, 253, 1563, 5981, 27, 10360, 1320, 27, 2325, 12287, 654, 58, 11862, 14, 13888, 14, 41, 8147, 13208]}


2it [00:01,  1.11it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

i Evaluating: {'question': 'Are there any code samples illustrating how to implement custom logging handlers?', 'answer': 'Yes, the Python logging module documentation provides several examples of how to implement custom logging handlers. You can find them in the official documentation here: https://docs.python.org/3/howto/logging-cookbook.html#developing-new-handlers', 'input_ids': [6723, 627, 667, 2127, 3530, 34805, 849, 281, 3359, 2840, 20893, 40093, 32, 4374, 13, 253, 13814, 20893, 6333, 10097, 3400, 2067, 6667, 273, 849, 281, 3359, 2840, 20893, 40093, 15, 1422, 476, 1089, 731, 275, 253, 3565, 10097, 1060, 27, 5987, 1358, 13880, 15, 16659, 15, 2061, 16, 20, 16, 5430, 936, 16, 36193, 14, 29519, 3305, 15, 2974, 4, 16714, 272, 14, 1826, 14, 4608, 10787], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': 

11it [00:09,  1.18it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


i Evaluating: {'question': 'Are there any code samples illustrating how to handle authentication and authorization?', 'answer': 'Yes, there is a separate section in the documentation explaining authentication, for more information visit https://lamini-ai.github.io/auth/', 'input_ids': [6723, 627, 667, 2127, 3530, 34805, 849, 281, 6016, 19676, 285, 26239, 32, 4374, 13, 627, 310, 247, 4858, 2593, 275, 253, 10097, 15571, 19676, 13, 323, 625, 1491, 4143, 5987, 1358, 77, 4988, 74, 14, 2284, 15, 7280, 15, 900, 16, 14399, 16], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [6723, 627, 667, 2127, 3530, 34805, 849, 281, 6016, 19676, 285, 26239, 32, 4374, 13, 627, 310, 247, 4858, 2593, 275, 253, 10097, 15571, 19676, 13, 323, 625, 1491, 4143, 5987, 1358, 77, 4988, 74, 14, 2284, 15, 7280, 15, 900, 16, 14399, 16]}


11it [00:10,  1.07it/s]

Number of exact matches:  0


as we can see exact matches are 0 here.

In [24]:
df = pd.DataFrame(predictions, columns=["predicted_answer", "target_answer"])
print(df)

                                     predicted_answer  \
0   Yes, Lamini can generate technical documentati...   
1   You can use the Authorization HTTP header to g...   
2   Lamini’s LLM Engine is a LLM Engine for develo...   
3   Yes, there is a community or support forum ava...   
4   Yes, the Lamini library can be utilized for te...   
5   Lamini is designed to be flexible and scalable...   
6   You can instantiate the LLM engine using the L...   
7   Yes, Lamini provides mechanisms for compressio...   
8   The performance of LLMs trained using Lamini c...   
9   Yes, there is a support and community availabl...   
10  Yes, there are some code samples available in ...   
11  Yes, there are some code samples available tha...   

                                        target_answer  
0   Yes, Lamini can generate technical documentati...  
1   The Authorization HTTP header should include t...  
2   Yes, the code includes a version parameter in ...  
3   Yes, there is a community foru

In [25]:
evaluation_dataset_path = "lamini/lamini_docs_evaluation"
evaluation_dataset = datasets.load_dataset(evaluation_dataset_path)

Generating train split: 100%|██████████| 139/139 [00:00<00:00, 5779.11 examples/s]


In [26]:
pd.DataFrame(evaluation_dataset)

,train
0,"{'predicted_answer': 'Yes, Lamini can generate..."
1,{'predicted_answer': 'You can use the Authoriz...
2,{'predicted_answer': 'Lamini’s LLM Engine is a...
3,"{'predicted_answer': 'Yes, there is a communit..."
4,"{'predicted_answer': 'Yes, the Lamini library ..."
...,...
134,"{'predicted_answer': 'Yes, Lamini has the abil..."
135,{'predicted_answer': 'I wish! This documentati...
136,"{'predicted_answer': 'Yes, Lamini can generate..."
137,"{'predicted_answer': 'Yes, the documentation h..."


Academic benchmark ARC one of 4 eleutherAI came from

In [29]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [31]:
# !python lm-evaluation-harness/main.py --model hf-causal --model_args pretrained = lamini/lamini_docs_finetuned --tasks arc_easy -- device cpu

This Arc benchmark evaluation technique can be used for finding the base model but not for actual fine tunning task.

Practical approach to finetuning

1. Figure out your task
2. Collect data related to the task's inputs/outputs
3. Generate data if you don't have enough data use prompt template to create some more
4. Finetune a small model (eg. 400M-1B)
5. Vary the amount of data you give the model
6. Evaluate your LLM to know what's going well vs. not
7. collect more data to improve
8. Increase task complexity
9. Increase model size for performance


Complexity: More tokens out is harder

1. Extract: "reading" is easier     ------- Smaller models
    keywords, topics, routing, agents
2. Expand: "Writing" is harder     ------- Larger Models
    chat, write emails, write code

Combination of tasks is harder than 1 task

harder or more general = larger model

Eg. you might want an agent to be flexible and do serval things at once or in one step


You have model sizes but you also required compute requirements basically around hardware of what you need to run your models.

| AWS Instance | GPUs | GPU Memory | Max inference size(# of parameters) | Max training size (# of tokens) |
|-|-|-|-|-|
| p2.2xlarge    | 1 V100 | 16GB         | 7B  | 1B  |
| p3.8xlarge    | 4 V100 | 64GB         | 7B  | 1B  |
| p3.16xlarge   | 8 V100 | 128GB        | 7B  | 1B  |
| p3dn.24xlarge | 8 V100 | 256GB        | 14B | 2B  |
| p4d.24xlarge  | 8 A100 | 320GB HBM2   | 18B | 2.5B|
| p4de.24xlarge | 8 A100 | 640GB HBM2e  | 32B | 5B  |

PEFT : -Parameter-Efficient Finetuning

is set of different methods that help you do just that, be much more efficient in how youre using your parameters and training models. 

LoRa stands for Low rank adaptation

LoRa : - Low Rank Adaptation of LLms

1. Fewer trainable parameters: for GPT3, 10000x less

2. Less GPU memory: for GPT3 3x less

3. Slightly below accuracy to finetuning

4. Same inference latency

What actually happening in LoRa:- 

Train new weights in some layers, freeze main weights.

    New weights: rank decomposition matrices of original weights's change

    we can train new weights seperately, alternatively to the pretrained weights, but then at inference time be able to merge them back into the main pre-trained weights

    and get that fine-tuned model more efficiently.

    At inference, merge with main weights

User LoRa for adapting to new, different tasks, ie train a model using LoRa on one customer's data and then train another one on another customer's data and 

then be able to merge them each in at inference time when you need them.